# Лабораторная работа 2 : «Наивный» байесовский классификатор

In [ ]:
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda
from sklearn.linear_model import LogisticRegression as lr
from sklearn import svm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import itertools
import sklearn
from scipy import linalg as npl
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import ConfusionMatrixDisplay as cmd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import OrdinalEncoder

## Задание 1

In [ ]:
# V = 83 => вариант 12

df = pd.read_csv('mushrooms.csv')
df = df[['stalk-surface-above-ring', 'stalk-surface-below-ring','stalk-color-above-ring', 'stalk-color-below-ring','veil-type','class']]
X = df.drop(columns = ['class'])
y = df['class']
df

In [ ]:
for i in X:
    sns.catplot(x = i, data = df, hue = 'class',kind = 'count')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2022)

## Задание 2

In [ ]:
def GetProbByColumn(X_, y_, column, j):
    prob  = {}
    x = X_[column]
    x = x[y_==j]
    unique = X_[column].unique()
    k = len(unique)
    for i in unique:
        prob[i] = (x[x==i].count()) / (x.count())
    return prob

def Predict2(X_, y_, column):
    yPred = []
    prob0 = GetProbByColumn(X_, y_, column, 'e')
    prob1 = GetProbByColumn(X_, y_, column, 'p')
    c = y_.count()
    p0 = y_[y_=='e'].count() / c
    p1 = y_[y_=='p'].count() / c
    for k in X_[column]:
        if p0 * prob0[k] < p1 * prob1[k]:
            yPred.append('p')
        else:
            yPred.append('e')
    return np.array(yPred)

In [ ]:
G = pd.DataFrame(columns = ['score_test','score_train'])
for col in X.columns:
    G = G.append({'score_test' : accuracy_score(y_test, Predict2(X_test, y_test, col)), 'score_train' : accuracy_score(y_train, Predict2(X_train, y_train, col))},ignore_index=True)
G = G.set_index(X.columns)
G

видно, что самая информативная переменная - **stalk-surface-above-ring**.

## Задание 3

In [ ]:
X_train_ohe = pd.get_dummies(X_train)
X_test_ohe = pd.get_dummies(X_test)
y_train_ohe = y_train
y_test_ohe = y_test

model = sklearn.naive_bayes.BernoulliNB()
model.fit(X_train_ohe,y_train_ohe)
score = model.score(X_test_ohe,y_test_ohe)
print('Score:', score)

## Задание 4

In [ ]:
def pred(X_, col, p, q, P1, P2): 
    l_0 = 0
    b_0 = 0
    for j in X_.columns:
        b_j = 0
        b_j = np.log(p[j]*(1-q[j])/(q[j]*(1-p[j])))
        l_0 += b_j*col[j]
        b_0 += np.log((1-p[j])/(1-q[j]))
    b_0 += np.log(P1/P2)
    return (l_0 + b_0)

def Predict4(X_, y_, X_tr, y_tr, useReg = False):
    p = {}
    q = {}
    y_tr_len = len(y_tr)
    P1 = len(y_tr[y_tr=='p'])/y_tr_len
    P2 = len(y_tr[y_tr=='e'])/y_tr_len
    for j in X_tr.columns:
        if (useReg):
            p[j] = (X_tr[y_tr=='p'][j].sum()+1)/(len(y_tr[y_tr=='p'])+2)
            q[j] = (X_tr[y_tr=='e'][j].sum()+1)/(len(y_tr[y_tr=='e'])+2)
        else:
            p[j] = (X_tr[y_tr=='p'][j].mean())
            q[j] = (X_tr[y_tr=='e'][j].mean())
        if(q[j] == 1):
            q[j] = 1.- 1e-10
        if(p[j] == 1):
            p[j] = 1.- 1e-10
        if(q[j] == 0):
            q[j] = 1e-10
        if(p[j] == 0):
            p[j] = 1e-10
    a = []
    for i in X_.index:
        a.append(pred(X_, X_.loc[i], p, q, P1, P2))
    b = []
    for i in a:
        if(i >= 0):
            b.append('p')
        else:
            b.append('e')
    b = np.array(b)
    err = 0
    for i in range(len(b)):
        if(b[i] != y_.iloc[i]):
            err += 1
    score = 1 - err / len(b)
    return score

In [ ]:
print('Score:', Predict4(X_test_ohe, y_test_ohe, X_train_ohe, y_train_ohe))

добавим теперь регуляризатор:

In [ ]:
print('Score:', Predict4(X_test_ohe, y_test_ohe, X_train_ohe, y_train_ohe, True))

результат не изменился.

Видно, что результаты совпадают с библиотечным методом.